### 注意力评分函数


假设有查询$\mathbf{q} \in \mathbb{R}^q$和m个Key-Value Pair
$(\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m,\mathbf{v}_m)$,其中$\mathbb{k_i} \in \mathbb{R^k},\mathbf{v_i} \in \mathbb{R^v}$注意力汇聚函数$f$就被表示成值的加权和:
$$f(\mathbf{q}, (\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)) = \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i \in \mathbb{R}^v,$$
其中,$\mathbf{q}和\mathbf{k_i}$的注意力权重通过注意力评分函数$a$将两个向量映射成标量,再经过softmax运算得到:
$$
\alpha(\mathbf{q},\mathbf{k}_i)=\mathrm{softmax}(a(\mathbf{q},\mathbf{k}_i)) = \frac {\exp(a(\mathbf{q},\mathbf{k}_j))}{\sum_{j=1}^m \exp(a(\mathbf{q},\mathbf{k}_j))} \in \mathbb{R}
$$


不同的注意力评分函数会导致不同的注意力汇聚操作.

In [2]:
import math
import torch
from torch import nn
import sys,os
sys.path.append(os.path.abspath("../"))
import lmy
from lmy import show_heatmaps
import d2l


#### 掩码softmax操作
目的:删除部分无意义的,长度超限的元素,进行softmax操作.

In [7]:
def masked_softmax(X,valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X,dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens,shape[1])
        else:
            valid_lens = valid_lens.reshape(-1) 
        X = d2l.sequence_mask(X.reshape(-1,shape[-1]),valid_lens,value=-1e6)
        X= X.reshape(shape)
        print(X)
        return nn.functional.softmax(X,dim=-1)

In [15]:
masked_softmax(torch.arange(1, 5,dtype=torch.float32), torch.tensor(3))

tensor([ 1.0000e+00,  2.0000e+00,  3.0000e+00, -1.0000e+06])


tensor([0.0900, 0.2447, 0.6652, 0.0000])

In [22]:
# 效果与上述相同
X = torch.arange(1,4,dtype=torch.float32)
X,X.softmax(-1)

(tensor([1., 2., 3.]), tensor([0.0900, 0.2447, 0.6652]))

### 加性注意力
一般来说,当Query和Key是不同长度的矢量时,我们可以使用加性注意力作为评分函数.给定一个查询$q \in \mathbb{R}^q$和键$k \in \mathbb{R}^k$,additive attention 的评分函数为
$$
a(q,k) = \mathbf{w}_v^T\mathrm{tanh}(\mathbf{w}_q q+\mathbf{w}_k k ) \in \mathbb{R}
$$
,其中可学习的参数是wq和wk,wv.将查询和键连接到一起后输入到一个MLP中,感知机包含一个隐含层,单元数量为超参数h,使用tanh作为激活函数且禁止偏置项.

In [ ]:
class AdditiveAttention(nn.Module):
    

## 